In [2]:
# !pip install -q ipython-autotime
# %load_ext autoreload
# %load_ext autotime
# %autoreload 2

In [3]:
import requests, os, tarfile
from tqdm import tqdm
from data_download import download, extract_tgz

dataset_url = r'https://www.robots.ox.ac.uk/~vgg/data/flowers/102/102flowers.tgz'
segmentations = r'https://www.robots.ox.ac.uk/~vgg/data/flowers/102/102segmentations.tgz'
chi_distance = r'https://www.robots.ox.ac.uk/~vgg/data/flowers/102/distancematrices102.mat'
labels = r'https://www.robots.ox.ac.uk/~vgg/data/flowers/102/imagelabels.mat'
splits = r'https://www.robots.ox.ac.uk/~vgg/data/flowers/102/setid.mat'

OVERWRITE = False

download(dataset_url, dataset_url.split('/')[-1], force_redownload=OVERWRITE)
# download(segmentations, segmentations.split('/')[-1], force_redownload=OVERWRITE)
# download(chi_distance, chi_distance.split('/')[-1], force_redownload=OVERWRITE)
download(labels, labels.split('/')[-1], force_redownload=OVERWRITE)
download(splits, splits.split('/')[-1], force_redownload=OVERWRITE)
extract_tgz(r'102flowers.tgz')

File already exists
File already exists
File already exists
File already extracted


In [4]:
import torch
import os
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
from PIL import Image
from scipy.io import loadmat
import numpy as np
import pandas as pd
from data_processing import prepare_df, FlowerDataset


split_path = r"data/setid.mat"
labels_Path = r"data/imagelabels.mat"
data_root = r"data/102flowers/jpg"
train_split, test_split, val_split = prepare_df(split_path, labels_Path, data_root)

transformsations = transforms.Compose(
    [
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
    ]
)

# train_dataset = FlowerDataset(train_split, transform=transformsations)
# test_dataset = FlowerDataset(test_split, transform=transformsations)
# val_dataset = FlowerDataset(val_split, transform=transformsations)


# train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
# val_loader = DataLoader(val_dataset, batch_size=32, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
# train_split

In [5]:
import torchvision
import torchsummary
import torch.nn as nn
import segmentation_models_pytorch as smp



In [6]:
def accuray_fn(ypred, y_true):
    """
    Calculate the accuracy of the prediction
    """
    return (ypred.argmax(dim=1) == y_true).sum() / len(ypred)


In [7]:
import torch
from models  import Resnet50Flower102
from tqdm import tqdm
device = "cuda" if torch.cuda.is_available() else "cpu"
# device = 'cpu'
model = Resnet50Flower102(pretrained=True).to(device)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
# optimizer = torch.optim.Adam(model.model.fc.parameters(), lr=3e-4)
train_dataset = FlowerDataset(train_split, transform=transformsations)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

c:\Users\DELL\anaconda3\envs\torch\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [8]:
# from torch.utils.tensorboard import SummaryWriter
# writer = SummaryWriter()


# train_loader = DataLoader(train_dataset, batch_size=4, shuffle=False)
# EPOCHS = 10

# for epoch in range(EPOCHS):
#     total_loss = 0
#     total_acc = 0
#     for x, y in tqdm(
#         train_loader, total=len(train_loader), desc="Training", leave=False
#     ):
#         x, y = x.to(device), y.to(device)
        
#         optimizer.zero_grad()
#         y_hat = model(x)
#         loss = loss_fn(y_hat, y)
#         loss.backward()
#         optimizer.step()
#         total_loss += loss.item()
#         total_acc += accuray_fn(y_hat, y)

#     # writer.add_scalar("Loss/train", total_loss / len(train_loader), epoch)
#     # writer.add_scalar("Accuracy/train", total_acc / len(train_loader), epoch)

#     print(f"Epoch {epoch} Loss : {total_loss / len(train_loader)}")
#     print(f"Epoch {epoch} Accuracy: {total_acc/len(train_loader)}")

# writer.flush()

In [9]:
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=False)
EPOCHS = 10

for epoch in range(EPOCHS):
    total_loss = 0
    total_acc = 0
    for i in range(10):
        x, y = next(iter(train_loader))
        x, y = x.to(device), y.to(device)
        optimizer.zero_grad()
        y_hat = model(x)
        loss = loss_fn(y_hat, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        total_acc += accuray_fn(y_hat, y)

    # writer.add_scalar("Loss/train", total_loss / len(train_loader), epoch)
    # writer.add_scalar("Accuracy/train", total_acc / len(train_loader), epoch)

    print(f"Epoch {epoch} Loss : {total_loss / len(train_loader)}")
    print(f"Epoch {epoch} Accuracy: {total_acc/len(train_loader)}")

    

Epoch 0 Loss : 0.003039207818139204
Epoch 0 Accuracy: 0.0058517553843557835
Epoch 1 Loss : 0.0
Epoch 1 Accuracy: 0.0065019503235816956
Epoch 2 Loss : 0.0
Epoch 2 Accuracy: 0.0065019503235816956
Epoch 3 Loss : 0.0
Epoch 3 Accuracy: 0.0065019503235816956
Epoch 4 Loss : 0.0
Epoch 4 Accuracy: 0.0065019503235816956
Epoch 5 Loss : 0.0
Epoch 5 Accuracy: 0.0065019503235816956
Epoch 6 Loss : 0.0
Epoch 6 Accuracy: 0.0065019503235816956
Epoch 7 Loss : 0.0
Epoch 7 Accuracy: 0.0065019503235816956
Epoch 8 Loss : 0.0
Epoch 8 Accuracy: 0.0065019503235816956
Epoch 9 Loss : 0.0
Epoch 9 Accuracy: 0.0065019503235816956
